In [1]:
import pandas as pd
from typing import Tuple, List
import re
from datetime import datetime
import io

def read_bat_file(file_path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Reads a .bat file containing structured data and returns two pandas DataFrames.
    
    Args:
        file_path (str): Path to the .bat file
        
    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: 
            - First DataFrame: DocumentsOfRecord records
            - Second DataFrame: DocumentAttachment records
    """
    
    def parse_line(line: str) -> List[str]:
        """Parse a pipe-separated line into fields"""
        return [field.strip() for field in line.split('|')]
    
    def process_records(lines: List[str]) -> pd.DataFrame:
        """Convert list of record lines into a DataFrame"""
        if not lines:
            return pd.DataFrame()
        
        # First line is the header
        headers = parse_line(lines[0])
        
        # Remaining lines are data
        data_rows = []
        for line in lines[1:]:
            if line.strip():  # Skip empty lines
                row_data = parse_line(line)
                # Ensure row has same number of columns as headers
                while len(row_data) < len(headers):
                    row_data.append('')
                data_rows.append(row_data[:len(headers)])  # Truncate if too long
        
        return pd.DataFrame(data_rows, columns=headers)
    
    # Read the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Clean lines (remove newlines and empty lines)
    lines = [line.strip() for line in lines if line.strip()]
    
    # Group lines by record type
    record_groups = {}
    current_type = None
    current_lines = []
    
    for line in lines:
        # Check if this is a header line (starts with FILENAME|METADATA)
        if line.startswith('FILENAME|METADATA|'):
            # Save previous group if exists
            if current_type and current_lines:
                record_groups[current_type] = process_records(current_lines)
            
            # Start new group
            parts = parse_line(line)
            if len(parts) > 2:
                current_type = parts[2]  # DocumentsOfRecord or DocumentAttachment
                current_lines = [line]
        else:
            # This is a data line
            if current_type:
                current_lines.append(line)
    
    # Don't forget the last group
    if current_type and current_lines:
        record_groups[current_type] = process_records(current_lines)
    
    # Return the two DataFrames as a tuple
    documents_of_record_df = record_groups.get('DocumentsOfRecord', pd.DataFrame())
    document_attachment_df = record_groups.get('DocumentAttachment', pd.DataFrame())
    
    return documents_of_record_df, document_attachment_df

def normalize_name(name: str) -> str:
    """Normalize name for comparison - handle case and spacing"""
    if pd.isna(name) or name == '' or str(name).upper() == 'NESSUN DIPENDENTE':
        return ''
    return str(name).strip().upper()

def normalize_date(date_str: str) -> str:
    """Normalize date string for comparison"""
    if pd.isna(date_str) or date_str == '' or str(date_str).upper() == 'NESSUNA DATA':
        return ''
    
    # Remove any extra whitespace
    date_str = str(date_str).strip()
    
    # Handle different date formats
    date_patterns = [
        r'(\d{1,2})/(\d{1,2})/(\d{4})',  # DD/MM/YYYY or D/M/YYYY
        r'(\d{4})/(\d{1,2})/(\d{1,2})',  # YYYY/MM/DD or YYYY/M/D
        r'(\d{4})-(\d{1,2})-(\d{1,2})',  # YYYY-MM-DD
    ]
    
    for pattern in date_patterns:
        match = re.search(pattern, date_str)
        if match:
            if pattern == date_patterns[0]:  # DD/MM/YYYY format
                day, month, year = match.groups()
                return f"{int(day):02d}/{int(month):02d}/{year}"
            elif pattern == date_patterns[1]:  # YYYY/MM/DD format
                year, month, day = match.groups()
                return f"{int(day):02d}/{int(month):02d}/{year}"
            elif pattern == date_patterns[2]:  # YYYY-MM-DD format
                year, month, day = match.groups()
                return f"{int(day):02d}/{int(month):02d}/{year}"
    
    return date_str.upper()

def extract_name_parts(full_name: str) -> Tuple[str, str]:
    """Extract first name and last name from full name (FIRST_NAME LAST_NAME format)"""
    if pd.isna(full_name) or full_name == '' or str(full_name).upper() == 'NESSUN DIPENDENTE':
        return '', ''
    
    # Convert to uppercase and split
    full_name_upper = str(full_name).strip().upper()
    parts = full_name_upper.split()
    
    if len(parts) >= 2:
        # Format is FIRST_NAME LAST_NAME
        first_name = parts[0]  # First part is first name
        last_name = ' '.join(parts[1:])  # Rest is last name
        return first_name, last_name
    elif len(parts) == 1:
        # Only one name - treat as first name
        return parts[0], ''
    else:
        return '', ''

def compare_dataframes(bat_file_path: str, csv_data) -> pd.DataFrame:
    """
    Compare BAT file data with CSV data and count exact matches
    
    Args:
        bat_file_path (str): Path to the BAT file
        csv_data: Either a string (tab-separated CSV data) or a pandas DataFrame
        
    Returns:
        pd.DataFrame: Comparison results with match counts
    """
    
    # Read BAT file
    documents_df, attachments_df = read_bat_file(bat_file_path)
    
    # Combine both DataFrames from BAT file
    bat_records = []
    
    # Add DocumentsOfRecord entries
    if not documents_df.empty and 'DocumentName' in documents_df.columns:
        for _, row in documents_df.iterrows():
            bat_records.append({
                'filename': row.get('FILENAME', ''),
                'full_name': row.get('DocumentName', ''),
                'date': row.get('DateFrom', ''),
                'source': 'DocumentsOfRecord'
            })
    
    # Add DocumentAttachment entries if they have name info
    if not attachments_df.empty and 'Title' in attachments_df.columns:
        for _, row in attachments_df.iterrows():
            # Try to extract name from Title or other fields
            title = row.get('Title', '')
            if title and title != row.get('FILENAME', ''):
                bat_records.append({
                    'filename': row.get('FILENAME', ''),
                    'full_name': title,
                    'date': '',  # Attachments might not have dates
                    'source': 'DocumentAttachment'
                })
    
    # Read CSV data - handle both string and DataFrame inputs
    if isinstance(csv_data, pd.DataFrame):
        csv_df = csv_data
    else:
        # Assume it's a string with tab-separated data
        csv_df = pd.read_csv(io.StringIO(csv_data), sep='\t')
    
    # Prepare comparison results
    results = []
    
    for csv_idx, csv_row in csv_df.iterrows():
        csv_filename = str(csv_row.get('Nome file', ''))
        csv_full_name = str(csv_row.get('Nominativo', ''))
        csv_date = str(csv_row.get('Data', ''))
        csv_cluster = str(csv_row.get('Cluster', ''))
        
        csv_first_name, csv_last_name = extract_name_parts(csv_full_name)
        csv_norm_date = normalize_date(csv_date)
        
        # Count matches for this CSV row
        filename_matches = 0
        first_name_matches = 0
        last_name_matches = 0
        date_matches = 0
        full_matches = 0
        
        matching_bat_records = []
        
        for bat_record in bat_records:
            bat_filename = bat_record['filename']
            bat_full_name = bat_record['full_name']
            bat_date = bat_record['date']
            
            bat_first_name, bat_last_name = extract_name_parts(bat_full_name)
            bat_norm_date = normalize_date(bat_date)
            
            # Check individual matches
            filename_match = csv_filename.lower() == bat_filename.lower() if csv_filename and bat_filename else False
            first_name_match = csv_first_name == bat_first_name if csv_first_name and bat_first_name else False
            last_name_match = csv_last_name == bat_last_name if csv_last_name and bat_last_name else False
            date_match = csv_norm_date == bat_norm_date if csv_norm_date and bat_norm_date else False
            
            # Count matches
            if filename_match:
                filename_matches += 1
            if first_name_match:
                first_name_matches += 1
            if last_name_match:
                last_name_matches += 1
            if date_match:
                date_matches += 1
            
            # Full match: first name, last name, and date all match
            if first_name_match and last_name_match and date_match:
                full_matches += 1
                matching_bat_records.append(f"{bat_full_name} ({bat_norm_date})")
        
        results.append({
            'CSV_Row': csv_idx,
            'CSV_Filename': csv_filename,
            'CSV_Name': csv_full_name,
            'CSV_Date': csv_date,
            'CSV_Cluster': csv_cluster,
            'CSV_First_Name': csv_first_name if csv_first_name else '',
            'CSV_Last_Name': csv_last_name if csv_last_name else '',
            'CSV_Normalized_Date': csv_norm_date,
            'Filename_Matches': filename_matches,
            'First_Name_Matches': first_name_matches,
            'Last_Name_Matches': last_name_matches,
            'Date_Matches': date_matches,
            'Full_Matches': full_matches,
            'Matching_Records': '; '.join(matching_bat_records) if matching_bat_records else ''
        })
    
    results_df = pd.DataFrame(results)
    
    # Calculate overall statistics
    total_rows = len(results_df)
    total_possible_matches = total_rows * 3  # 3 fields per row: first_name, last_name, date
    
    # Count actual matches (only count if the field had valid data to match)
    actual_first_name_matches = sum(1 for r in results if r['CSV_First_Name'] != '' and r['First_Name_Matches'] > 0)
    actual_last_name_matches = sum(1 for r in results if r['CSV_Last_Name'] != '' and r['Last_Name_Matches'] > 0)
    actual_date_matches = sum(1 for r in results if r['CSV_Normalized_Date'] != '' and r['Date_Matches'] > 0)
    
    total_actual_matches = actual_first_name_matches + actual_last_name_matches + actual_date_matches
    
    # Count valid fields (fields that had data to potentially match)
    valid_first_names = sum(1 for r in results if r['CSV_First_Name'] != '')
    valid_last_names = sum(1 for r in results if r['CSV_Last_Name'] != '')
    valid_dates = sum(1 for r in results if r['CSV_Normalized_Date'] != '')
    total_valid_fields = valid_first_names + valid_last_names + valid_dates
    
    # Add summary statistics to the dataframe as metadata
    results_df.attrs['summary'] = {
        'total_rows': total_rows,
        'total_possible_matches': total_possible_matches,
        'total_actual_matches': total_actual_matches,
        'total_valid_fields': total_valid_fields,
        'match_rate_vs_possible': total_actual_matches / total_possible_matches if total_possible_matches > 0 else 0,
        'match_rate_vs_valid': total_actual_matches / total_valid_fields if total_valid_fields > 0 else 0,
        'valid_first_names': valid_first_names,
        'valid_last_names': valid_last_names,
        'valid_dates': valid_dates,
        'matched_first_names': actual_first_name_matches,
        'matched_last_names': actual_last_name_matches,
        'matched_dates': actual_date_matches
    }
    
    return results_df

def print_match_summary(comparison_results: pd.DataFrame):
    """
    Print a summary of the matching statistics
    
    Args:
        comparison_results (pd.DataFrame): Results from compare_dataframes function
    """
    summary = comparison_results.attrs.get('summary', {})
    
    print("=== MATCHING SUMMARY ===")
    print(f"Total rows analyzed: {summary.get('total_rows', 0)}")
    print(f"Total possible matches (rows × 3 fields): {summary.get('total_possible_matches', 0)}")
    print(f"Total actual matches found: {summary.get('total_actual_matches', 0)}")
    print(f"Total valid fields (with data): {summary.get('total_valid_fields', 0)}")
    print()
    print("MATCH RATES:")
    print(f"  Against all possible (matches/rows×3): {summary.get('match_rate_vs_possible', 0):.2%}")
    print(f"  Against valid fields only: {summary.get('match_rate_vs_valid', 0):.2%}")
    print()
    print("FIELD BREAKDOWN:")
    print(f"  First Names: {summary.get('matched_first_names', 0)}/{summary.get('valid_first_names', 0)} matched")
    print(f"  Last Names: {summary.get('matched_last_names', 0)}/{summary.get('valid_last_names', 0)} matched")
    print(f"  Dates: {summary.get('matched_dates', 0)}/{summary.get('valid_dates', 0)} matched")
    print()
    print(f"SCORE: {summary.get('total_actual_matches', 0)}/{summary.get('total_possible_matches', 0)} " +
          f"({summary.get('match_rate_vs_possible', 0):.1%})")

# Example usage:
if __name__ == "__main__":
    # Option 1: CSV data from Excel file
    csv_data = pd.read_excel('Docs Train.xlsx', engine='openpyxl')  # Changed to .xlsx
    
    # Compare the dataframes
    comparison_results = compare_dataframes('DocumentsOfRecord.dat', csv_data)
    
    # Print summary statistics
    print_match_summary(comparison_results)
    
    # Display detailed results
    print("\nDetailed Results:")
    print(comparison_results[['CSV_Name', 'First_Name_Matches', 'Last_Name_Matches', 'Date_Matches', 'Full_Matches']])
    
    # Option 2: If you have a CSV file instead
    # csv_data = pd.read_csv('extracted_data.csv')
    # comparison_results = compare_dataframes('your_bat_file.bat', csv_data)
    
    # Option 3: Original string format still works
    # csv_string = """Nome file	Cluster	Nominativo	Data
    # 0004830357001_2008893.TIF	Nessun cluster	Sassi Maurizio	18/06/1984"""
    # comparison_results = compare_dataframes('your_bat_file.bat', csv_string)

=== MATCHING SUMMARY ===
Total rows analyzed: 63
Total possible matches (rows × 3 fields): 189
Total actual matches found: 57
Total valid fields (with data): 185

MATCH RATES:
  Against all possible (matches/rows×3): 30.16%
  Against valid fields only: 30.81%

FIELD BREAKDOWN:
  First Names: 1/62 matched
  Last Names: 1/62 matched
  Dates: 55/61 matched

SCORE: 57/189 (30.2%)

Detailed Results:
              CSV_Name  First_Name_Matches  Last_Name_Matches  Date_Matches  \
0       Sassi Maurizio                   0                  0             1   
1   Paltrinieri Silvia                   0                  0             1   
2       Cioni Giuseppe                   0                  0             1   
3         Poppi Andrea                   0                  0             1   
4   Ficarazzo Riccardo                   0                  0             1   
..                 ...                 ...                ...           ...   
58       Busi Leonardo                   0       